In [2]:
import pandas as pd
import plotly_express as px
from hash_data import Hash_DataFrame as hd

# file path 
file_path = "Data/athlete_events.csv"

# load the data as a dataFrame 
df_data = pd.read_csv(file_path)

# encrypt the athletes names. TODO check why error message is not working.
df_data = hd.hash_Columns(df_data, ["Name"])

df_data


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,3a4eef48434c66b3f14ab0221f6762d0ef7c6135ab2790...,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,a6430cc6630934275dc6283f7e97e9625e6587cdddec7a...,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,9c198b205332c2c8e1542e0f9534b9e270780a41d978ec...,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,0a477bb1c5ad39716f9c775e54d18d16aa8b37ada55548...,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,5b7be356aa28178096dc6747f0b8e4e393eaceb5f95310...,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,07f491e055f56a7a7be133c9cf6563c253d3aa0d137630...,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,2f748d07ca91a5a1a6871b8f05cdd4d69f194615a82ca5...,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,2f748d07ca91a5a1a6871b8f05cdd4d69f194615a82ca5...,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,e7fd567d163933f9b76a0094f18afbf8e8291b179c5487...,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [3]:
# de sporter landet fått flest medaljer i

# get all swe data from dataframe
df_data_sweden = df_data.loc[df_data["Team"] == "Sweden"]

# sort out team sports
df_data_sweden = df_data_sweden.groupby(["Games", "Event", "Medal", "Team", "Year"]).sum(numeric_only= False).reset_index()

# groupby sport and count medals 
df_sport_medal = df_data_sweden[["Sport", "Medal"]].groupby("Sport").count().reset_index().sort_values(by= "Medal", ascending= False).head(10)

# NOTE i lag sport är det en medalj per lag medlem.
fig = px.bar(df_sport_medal, x= "Sport", y= "Medal")
fig.show()

In [4]:
# antal medaljer per OS

# Create a new dataframe with year and medals 
df_os_medals = df_data_sweden.groupby("Year")["Medal"].count().reset_index()

# rename Medal to toalal medals
df_os_medals.rename(columns= {"Medal": "total Medals"}, inplace= True)

# seasons and column names 
seasons = ["Summer", "Winter"]
column_names = ["Summer medals", "Winter medals"]

# counts medals for winter and summer os and add them to new columns "Summer medals" and "Winter medals"
for i, season in enumerate(seasons):

    df_temp = df_data_sweden[["Year","Medal"]].loc[df_data_sweden["Season"] == season].groupby("Year").count().reset_index()
    df_temp.rename(columns= {"Medal": column_names[i]}, inplace=True)
    df_os_medals = pd.merge(df_os_medals, df_temp, how= "left", on= "Year")

# Titels and column names 
titels = ["Total number of medals per OS", "Total number of medals per summer OS", "Total number of medals per winter OS"]
columns = [["Summer medals", "Winter medals"], "Summer medals", "Winter medals"]

# plots the graphs
for i, titel in enumerate(titels):

    fig = px.bar(df_os_medals, x= "Year", y=columns[i], barmode="group", title= titel)
    fig.update_layout(
        yaxis_title="number of medals"
    )
    fig.show()





In [5]:
# histogram över åldrar
# gets swedish participants
df_swedish_participants = df_data.loc[df_data["Team"] == "Sweden"]

# group the data where name is the same and creates a new dataframe with age and name.
df_age = df_swedish_participants[["Age", "Name"]].groupby(["Name", "Age"]).count().reset_index()

# Plots the histogram 
fig = px.histogram(df_age, x= "Age", title="OS age distribution in Sweden")
fig.update_layout(
        yaxis_title="Number"
    )
fig.show()

In [6]:
# Kön fördelning i os för sverige 

# group by sex and name
df_swe_sex = df_swedish_participants.groupby(["Sex", "Name"]).count().reset_index()

#group by sex. 
df_swe_sex = df_swe_sex.groupby("Sex")["Sex"].count()

# Plots graph
fig = px.pie(df_swe_sex, names= df_swe_sex.index, values= "Sex", title= "Gender")
fig.show()